__In this notebook, we collect, annotate and pre-process deep mutational scanning (DMS) and alanine scanning (AS) data for future analysis.__

In [1]:
import os
from scipy.stats import spearmanr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../src/')
import mavedb_tool as mtool
import preproc as preproc

Duplicate key in file '/Users/fu.j/.matplotlib/matplotlibrc', line 2 ('backend: TkAgg')


In [2]:
data_info = pd.read_csv('../data/data_info/data_compatibility_221024.csv', index_col=0)

# DMS data

## Download

We download all available DMS data from [MaveDB](https://www.mavedb.org/) by the time this study begins.

In [38]:
for urn_id in data_info.dms_id.unique():
    # Temporary, following data to be uploaded to MaveDB.
    if urn_id[:3] != 'urn':
        continue
    # We are going to use score.exp column instead of score column for these datasets. Explained below.
    if urn_id[-4:] == '-exp':
        urn_id = urn_id[:-4]
    #mtool.download_mavedb_score(urn_id, '../data/raw/mavedb_data/')

Downloading: urn:mavedb:00000069-a-1 from MaveDB.
Downloading: urn:mavedb:00000058-a-1 from MaveDB.
Downloading: urn:mavedb:00000003-a-2 from MaveDB.
Downloading: urn:mavedb:00000003-b-2 from MaveDB.
Downloading: urn:mavedb:00000097-0-1 from MaveDB.
Downloading: urn:mavedb:00000001-c-1 from MaveDB.
Downloading: urn:mavedb:00000005-a-5 from MaveDB.
Downloading: urn:mavedb:00000005-a-6 from MaveDB.
Downloading: urn:mavedb:00000047-a-1 from MaveDB.
Downloading: urn:mavedb:00000047-b-1 from MaveDB.
Downloading: urn:mavedb:00000047-c-1 from MaveDB.
Downloading: urn:mavedb:00000046-a-1 from MaveDB.
Downloading: urn:mavedb:00000051-a-1 from MaveDB.
Downloading: urn:mavedb:00000048-a-1 from MaveDB.
Downloading: urn:mavedb:00000048-b-1 from MaveDB.
Downloading: urn:mavedb:00000048-c-1 from MaveDB.
Downloading: urn:mavedb:00000062-b-1 from MaveDB.
Downloading: urn:mavedb:00000062-a-1 from MaveDB.
Downloading: urn:mavedb:00000077-a-1 from MaveDB.
Downloading: urn:mavedb:00000077-b-1 from MaveDB.


## Annotate

We further annotated the downloaded MaveDB data for the analysis of single amino acid variants (SAV).

In this analysis, we only use __experimental DMS scores__ which are usually stored in the `score` column of MaveDB data. But some studies may report __computationally imputed/refined scores__ instead, and store the experimental ones in `exp.score`. So, we swapped these columns in these cases.

For each variant entry:
1. If it is a __single amino acid missense variant__, the wild-type amino acid, variant type amino acid and location of substitution will be recorded
2. If it is a synonymous (or unchanged), nonsense or deletion variant, only the type (synonymous, ...) will be recorded
3. For multi-variants, frameshift or other types of variants, no detailed information will be recorded

In [7]:
annot_data = dict()
for dms_id in data_info.dms_id.unique():
    if dms_id[-4:] == '-exp':
        urn_id = dms_id[:-4]
        # We are going to use score.exp column instead of score column for these datasets.
        mave_data = pd.read_csv(f"../data/raw/mavedb_data/{urn_id}.csv", header=4)
        mave_data['score'] = mave_data['exp.score']
        mave_data = mave_data.drop(columns='exp.score')
    else:
        urn_id = dms_id
        mave_data = pd.read_csv(f"../data/raw/mavedb_data/{urn_id}.csv", header=4)
        if urn_id == 'urn:mavedb:00000097-0-1':  # Special hgvs_pro format.
            mave_data = mave_data[mave_data['hgvs_pro'].notna()]  # Remove non-coding variants.
            mave_data['hgvs_pro'] = mave_data.hgvs_pro.str[12:]
    
    mave_data = mtool.clean_mavedb_scores(mave_data)
    mave_data = mtool.annotate_mavedb_sav_data(mave_data)
    mave_data['dms_id'] = dms_id
    annot_data[dms_id] = mave_data

## Normalize

We normalize each DMS dataset by the scores of wildtype-like (wildtype or synonymous) and nonsense-like variants. Only single amino acid missense variant will be kept after this process.

__For wildtype-like scores__, we use the scores of synonymous variants in the data, if available. Otherwise, we referred to the paper, especially the method for score calculation, to identify the wildtype-like scores, as shown in the below cell.

__For nonsense-like scores__, it is defined as the median DMS scores for the 1% missense variants with the strongest loss of function in each DMS dataset, which are usually lower in scores (indicated as `negative`). But in some cases, according to the paper, high DMS scores may represent loss of protein functions (indicated as `positive`). We listed these datasets with positive-loss below for a correct normalisation.

In [8]:
defined_wt_scores = {'urn:mavedb:00000004-a-1': 0,  # Indicated by score calculation.
                     'urn:mavedb:00000010-a-1': 1,  # Defined by the paper.
                     'urn:mavedb:00000012-a-1': 0,  # Indicated by score calculation.
                     'urn:mavedb:00000012-a-2': 0,  # Indicated by score calculation.,
                     'urn:mavedb:00000012-a-3': 0,  # Indicated by score calculation.,
                     'urn:mavedb:00000012-a-4': 0,  # Indicated by score calculation.,
                     'urn:mavedb:00000012-a-5': 0,  # Indicated by score calculation.,
                     'urn:mavedb:00000012-a-6': 0,  # Indicated by score calculation.,
                     'urn:mavedb:00000041-a-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000041-b-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000053-a-2': 0,  # Defined by the paper.
                     'urn:mavedb:00000060-a-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000060-a-2': 0,  # Defined by the paper.
                     'urn:mavedb:00000062-b-1': 0.7050428,  # Referred from urn:mavedb:00000062-a-1
                     'urn:mavedb:00000077-b-1': 0,  # Indicated by score calculation.
                     'urn:mavedb:00000079-a-1': 0,  # Indicated by score calculation.
                     'urn:mavedb:00000086-a-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000086-b-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000086-c-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000086-d-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000086-e-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000086-f-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000086-g-1': 0,  # Defined by the paper.
                     'urn:mavedb:00000081-a-2': 0,  # Indicated by score calculation.
                     'tmp:UCrl0vQc6AQv4KH4': 1,  # Indicated by score calculation.
                     }

# Nonsense-like variants in the following DMS data have positive scores, either identified according 
# to assay setup or directly from scores of nonsense variants.
positive_loss = ['urn:mavedb:00000041-a-1', 'urn:mavedb:00000041-b-1', 'urn:mavedb:00000045-a-1', 
                 'urn:mavedb:00000045-b-1', 'urn:mavedb:00000045-c-1', 'urn:mavedb:00000045-d-1', 
                 'urn:mavedb:00000045-e-1', 'urn:mavedb:00000045-f-1', 'urn:mavedb:00000045-g-1', 
                 'urn:mavedb:00000045-h-1', 'urn:mavedb:00000045-i-1', 'urn:mavedb:00000045-j-1', 
                 'urn:mavedb:00000045-k-1', 'urn:mavedb:00000045-l-1', 'urn:mavedb:00000051-a-1', 
                 'urn:mavedb:00000051-b-1', 'urn:mavedb:00000060-a-1', 'urn:mavedb:00000060-a-2', 
                 'urn:mavedb:00000076-a-1', 'urn:mavedb:00000076-b-1', 'urn:mavedb:00000076-c-1', 
                 'urn:mavedb:00000076-e-1', 'urn:mavedb:00000051-c-1', 'urn:mavedb:00000058-a-1', 
                 'urn:mavedb:00000059-a-1', 'urn:mavedb:00000068-b-1']

In [9]:
normed_data = dict()
for dms_id, input_data in annot_data.items():
    # Theses data are specially normalised as shown below.
    if dms_id in ['urn:mavedb:00000105-a-1', 'urn:mavedb:00000054-a-1']: 
        continue
        
    # Set wildtype-like and nonsense-variant-like scores.
    wt_score = mtool.get_dms_wiltype_like_score(input_data)
    if np.isnan(wt_score):
        wt_score = defined_wt_scores[dms_id]
    if dms_id in positive_loss:
        non_score = 'positive'
    else:
        non_score = 'negative'
        
    data = input_data.query("mut_type == 'missense'").copy()
    data = preproc.normalize_dms_score(data, wt_score, non_score)
    normed_data[dms_id] = data

In [10]:
# In this dataset, the original scoring methods is not log-transformed but most other DMS data are.
dms_id = 'urn:mavedb:00000105-a-1'
input_data = annot_data[dms_id]

data = input_data.query("mut_type == 'missense'").copy()
data['score'] = np.log(data['score'])  # log-transformation.
wt_score = 0  # Indicated by score calculation (original: 1, log-transformed: 0).
non_score = 'negative'
data = preproc.normalize_dms_score(data, wt_score, non_score)
normed_data[dms_id] = data

In [11]:
# Wildtype score of this data is questionable. Corrected score is confirmed from the paper.
dms_id = 'urn:mavedb:00000054-a-1'
input_data = annot_data[dms_id]

wt_score = 0  # Corrected.
non_score = 'negative'

data = input_data.query("mut_type == 'missense'").copy()
data = preproc.normalize_dms_score(data, wt_score, non_score)
normed_data[dms_id] = data

## Position mapping

We then map each DMS residue to the reference protein sequence from UniProt.

To do this, we first curated the `UniProtKB ID` for each DMS target and added the rational `offset` between DMS variant position and reference sequence in `data_info`.

Residues conform with the reference sequence will be kept, adding its position in the reference sequence. Otherwise the residue will be removed with notice.

In [12]:
def get_uniprot_seq(uniprot_id):
    """Read UniProt sequence as pandas.Series format from reference folder.
    """
    with open(f"../data/reference/{uniprot_id}.fasta", 'r') as file:
        str_seq = file.read()
    str_seq = ''.join(str_seq.split('\n')[1:-1])
    # Adding 1 to make the positions start with 1 rather than 0.
    pro_seq = pd.Series(dict(zip(np.arange(len(str_seq))+1, str_seq)))
    return pro_seq


def get_uniprot_matched_dms_residues(dms_id, dms_seq, pro_seq):
    """
    The function reports mismatched DMS residues compared to the UniProt sequence. Then it returns 
    the matched DMS residues in a list.
    """
    matched_dms_seq = dms_seq.copy()  # This is for visualizing the final matched sequence.
    matched_res = []
    # Go through each DMS residue to check with reference sequence.
    for i, upos_res in enumerate(matched_dms_seq.index):
        if matched_dms_seq[upos_res] != pro_seq[upos_res]:
            print(f"Delete: Residue {upos_res}. DMS wildtype: {dms_seq[upos_res]}. UniProt wildtype: {pro_seq[upos_res]}")
            matched_dms_seq[upos_res] = '-'
        else:
            matched_res.append(upos_res)
    print('Matcheded DMS seq.: >',''.join(matched_dms_seq))
    return matched_res

In [13]:
unip_info = data_info.groupby('dms_id')[['uniprot_id', 'offset']].first()

mapped_data = dict()
for dms_id, input_data in normed_data.items():
    # Set UniProt positions.
    unip = unip_info.loc[dms_id]['uniprot_id']
    offset = int(unip_info.loc[dms_id]['offset'])
    input_data['uniprot_id'] = unip
    input_data['u_pos'] = input_data['position'].astype(int) + offset
    
    # Check if DMS indicated protein sequence conforms with UniProt sequence.
    pro_seq = get_uniprot_seq(unip)
    dms_seq = input_data.groupby('u_pos')['aa1'].first()
    if any(dms_seq != pro_seq.loc[dms_seq.index]):
        print(f"MISMATCH FOUND IN {dms_id}:")
        matched_res = get_uniprot_matched_dms_residues(dms_id, dms_seq, pro_seq)
        input_data = input_data[input_data['u_pos'].isin(matched_res)]  # Drop wildtype mismatched residues.
        print('*'*80)
    mapped_data[dms_id] = input_data

MISMATCH FOUND IN urn:mavedb:00000003-a-2:
Delete: Residue 175. DMS wildtype: R. UniProt wildtype: K
Matcheded DMS seq.: > DLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQ-TSVIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEF
********************************************************************************
MISMATCH FOUND IN urn:mavedb:00000003-b-2:
Delete: Residue 175. DMS wildtype: R. UniProt wildtype: K
Matcheded DMS seq.: > DLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQ-TSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEF
*******************************************************************

In most cases there are only one mismatched residue for each DMS dataset which may related to polymorphism.

Only two DMS datasets show more than one mismatched residues which are all at the start or end of the scanned sequences, that may related to experimental setup.

## Add DeMaSk features

Raw feature values were downloaded from [DeMaSk website](https://demask.princeton.edu/query/), using whole protein sequence.

In [14]:
demask_dms = dict()
for dms_id, input_data in mapped_data.items():
    unip = input_data.iloc[0]['uniprot_id']
    feat_data = pd.read_csv(f"../data/raw/protein_feature/demask/{unip}.txt")
    featured_data = pd.merge(input_data, feat_data, left_on=['u_pos', 'aa1', 'aa2'], 
                             right_on=['pos', 'WT', 'var'], how='left', 
                             suffixes=['', '_online_pred'])
    featured_data['sub_type'] = featured_data['aa1'] + featured_data['aa2']
    demask_dms[dms_id] = featured_data[['dms_id', 'position', 'aa2', 'aa1', 'sub_type', 'score', 
                                        'u_pos', 'uniprot_id', 'entropy', 'log2f_var', 'matrix']]
demask_dms = pd.concat(demask_dms.values(), ignore_index=True)
demask_dms = demask_dms.astype({'position': int})
#demask_dms.to_csv('../data/normalized/dms_DeMaSk_features_221024.csv')

## Add Envision features

Features used in Envision were downloaded from its [online toolkit](https://envision.gs.washington.edu/shiny/envision_new/). Protein data were excluded if their feature values were not available. 

In [15]:
envision_feature_map = {
    "AA1": "aa1",
    "AA2": "aa2",
    "WT_Mut": "wt_mut",
    "AA1_polarity": "aa1_polarity",
    "AA2_polarity": "aa2_polarity",
    "AA1_pI": "aa1_PI",
    "AA2_pI": "aa2_PI",
    "AA1_weight": "aa1_weight",
    "AA2_weight": "aa2_weight",
    "AA1_volume": "aa1vol",
    "AA2_volume": "aa2vol",
    "AA1_PSIC": "aa1_psic",
    "AA2_PSIC": "aa2_psic",
    "delta_PSIC": "delta_psic",
    "solvent_accessibility": "accessibility",
    "SecondaryStructure": "dssp_sec_str",
    "phi_psi_angles": "phi_psi_reg",
    "Uniprot": "uniprot_id",
    "position": "u_pos",
}

In [17]:
envision_dms = dict()
for dms_id, input_data in mapped_data.items():
    input_data = input_data[['dms_id', 'position', 'aa2', 'aa1', 'score', 'u_pos', 'uniprot_id']].copy()
    unip = input_data.iloc[0]['uniprot_id']
    
    try:
        feat_data = pd.read_csv(f"../data/raw/protein_feature/envision/{unip}_envisionData.csv", 
                                index_col=0, na_values=[''], keep_default_na=False, low_memory=False)
    except FileNotFoundError:
        print(f"{dms_id} ({unip}) has no Envision feature and will be removed.")
        continue
    feat_data = feat_data.rename(columns=envision_feature_map)
    # Drop not used columns.
    feat_data = feat_data.drop(columns=['X1', 'id2', 'Variant', 'evolutionary_coupling_prop', 
                                        'evolutionary_coupling_avg_norm', 'Envision_predictions'])
    feat_data = feat_data.astype({"u_pos": int})
    feat_data['wt_mut'] = feat_data['aa1'] + feat_data['aa2']
    
    featured_data = pd.merge(input_data, feat_data, left_on=['uniprot_id', 'u_pos', 'aa1', 'aa2'], 
                             right_on=['uniprot_id', 'u_pos', 'aa1', 'aa2'], how='left')
    envision_dms[dms_id] = featured_data
    
envision_dms = pd.concat(envision_dms.values(), ignore_index=True)
envision_dms = envision_dms.astype({'position': int})
#envision_dms.to_csv('../data/normalized/dms_Envision_features_221024.csv')

urn:mavedb:00000001-c-1 (P0DP23) has no Envision feature and will be removed.
urn:mavedb:00000077-a-1 (Q06851) has no Envision feature and will be removed.
urn:mavedb:00000077-b-1 (B8I7V0) has no Envision feature and will be removed.
urn:mavedb:00000063-a-1 (P0ABQ4) has no Envision feature and will be removed.
urn:mavedb:00000063-b-1 (P0ABQ4) has no Envision feature and will be removed.
urn:mavedb:00000090-a-1 (P31016) has no Envision feature and will be removed.
urn:mavedb:00000090-b-1 (P31016) has no Envision feature and will be removed.
urn:mavedb:00000076-a-1 (P40515) has no Envision feature and will be removed.
urn:mavedb:00000076-b-1 (P40515) has no Envision feature and will be removed.
urn:mavedb:00000076-c-1 (P40515) has no Envision feature and will be removed.
urn:mavedb:00000076-e-1 (P40515) has no Envision feature and will be removed.
urn:mavedb:00000011-a-1 (P02829) has no Envision feature and will be removed.
urn:mavedb:00000039-a-1 (P02829) has no Envision feature and wil

# AS data

We searched for alanine scanning studies with key phrase `(alanine scanning) OR (alanine scan)` together with protein names or identifiers with DMS data available.

Appropriate AS data were collected and curated if:
* they match the target protein
* they are in numeric format
* their wildtype-like and nonsense-like scores are available (tested or can be identified)
* they contain single alanine substitutions
* they are experimental rather than computational scores

Raw published values are curated in the `score` column for input data. For results presented in bar plots, we measured and stored the length of each bar. For results presented in Western blot, we used ImageJ to quantify the intensity for each entry. Residue positions are mapped and checked to UniProt sequences.

AS scores are normalized in a same way as DMS, except that the nonsense-like scores are defined from the experiment directly (more details in `note` column).

In [18]:
root = '../data/raw/low-throughput_data/'
norm_as = []
for file_name in os.listdir(root):
    if file_name == '.DS_Store':
        continue
    ascan_id = file_name[:-4]
    as_data = pd.read_csv(f"{root}{file_name}", index_col=0)
    
    nons = float(as_data.loc[as_data['aa1'] == 'non', 'score'])
    wts = float(as_data.loc[as_data['aa1'] == 'wt', 'score'])
    as_data = as_data[as_data['aa2'] == 'A']
    as_data = as_data.astype({'u_pos': int, 'position': int}).rename(columns={'score': 'AS_score'})
    as_data['AS_score'] = (as_data['AS_score'] - nons) / (wts - nons)  # Normalize.
    norm_as.append(as_data)
    
norm_as = pd.concat(norm_as, ignore_index=True)
#norm_as.to_csv('../data/normalized/as_data_221024.csv')

# DMS-AS correlation

We then calculate the Spearman's correlation between each DMS-AS pair.  

We first merge (map) each DMS and AS data by protein positions, and add information about the DMS-AS pair.

Then the correlation and regularized correlation values are calculate between __AS scores__ and __DMS scores of alanine substitutions__.

In [19]:
# Merge in a cartesian product way.
dms_all_as = pd.merge(demask_dms, norm_as[['uniprot_id', 'u_pos', 'Ascan_id', 'AS_score']], 
                      on=['uniprot_id', 'u_pos'], how='left')

# Identifier for each DMS+AS combined dataset.
dms_all_as['dmsa_id'] = dms_all_as['dms_id'] + '@' + dms_all_as['Ascan_id'].astype(str)
dms_all_as['Ascan_score_avail'] = ~dms_all_as['AS_score'].isna()

In [33]:
corr_stat = []
for dmsa, df in dms_all_as.groupby('dmsa_id'):
    if dmsa[-4:] == '@nan':  # No AS data available.
        continue
        
    df = df[df['aa2'] == 'A']
    row = dict()
    row['dmsa_id'] = dmsa
    row['n_ala'] = len(df)
    corr = spearmanr(df['score'], df['AS_score'])[0]
    if np.isnan(corr):
        # Impute missing value by 0 which is caused by only one or no alanine residue available.
        row['ala_score_correlation'] = 0.
    else:
        row['ala_score_correlation'] = corr
    row['regularized_correlation'] = (row['n_ala']-1) / (row['n_ala']+1) * row['ala_score_correlation']
    corr_stat.append(row)
    
# Right merge: some DMS and AS data have no overlapping residues (e.g.: urn:mavedb:00000003-a-2@brca1_yeast_growth),
# that should be removed.
data_corr = pd.merge(data_info, pd.DataFrame(corr_stat), on='dmsa_id', how='right', validate='1:1')
data_corr = data_corr.sort_values(['protein_name', 'dmsa_id']).reset_index(drop=True)
#data_corr.to_csv('../data/data_info/dms_as_info_221024.csv')